In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import json
import urllib
import double_dip
import re
from IPython.display import Markdown, display

# ↘ Start here! ↙


### Parameters:
- `days_from_now` -- an integer value between 0 and 4. This will dictate which showtimes are retrieved.
- `ads_mins` -- an integer value indicating how many minutes of ads you think are run before each movie. My experience with Cinestar says that ~ 10 minutes of ads is what you'll experience.
- `trailers_mins` -- an integer value indicating how many minutes of trailers you think will be run before each movie. This varies between Movies and Cinemas... I tried to set the default to something reasonable (10 minutes).
- `max_waiting_mins` -- an integer value indicating how long you're willing to wait between the (estimated) end of a film and and the start time of the next film. 
- `acceptable_overlap_mins` -- an integer value indicating how much overlap between the end of a film and the start of the next film is 'acceptable'. Set this value to something above 0 when you're willing to miss the last bit of a  film, or the start of another film.
- `movies_to_exclude` -- A string or list of strings giving partial matches of titles of movies you're _NOT_ interesting in seeing. E.g., `['Alles', 'Mars']` will partially match to _Alles Steht Kopf_ and _Der Marsianer - Rettet Mark Watney_, and thus both films will be excluded from any double dip considerations.
- `interesting_movies`-- A string or list of strings giving partial matches of titles of movies you _ARE_ interested in seeing.
- `all_interesting_movies_must_be_in_dip` -- a boolean indicating how calculated double dips should be filtered, in the presence of `interesting_movies`. E.g. if
    ```
    interesting_movies = ['Spec', 'Der']
    ```
    and `all_interesting_movies_must_be_in_dip=True`, then only dips that include matches for "Spec" and "Der" will be included. If `all_interesting_movies_must_be_in_dip=False` included dips only have to match at least one of "Spec", or "Der". 
    
### Notes
- Once you've tweaked your parameters, just goto the `Cell` menu and select `Run All`
- If you need to _re-run_ the program, ***do not*** re-`Run All`. Instead:
    1. re-run the cell with all the parameters
    2. goto the cell with `double_dip.find_all_double_dips` (the cell above the 'Double Dips' section), and re-run that
  
  If you don't do this, you'll have to wait around will the program REscrapes showtime information, which is needlessly redundant.


In [2]:
days_from_now = 3
ads_mins=10
trailers_mins=10
max_waiting_mins=45
acceptable_overlap_mins=0
movies_to_exclude=['Alle', 'Der'] #set to None to disable
interesting_movies=['Spec'] # set to None to disable
all_interesting_movies_must_be_in_dip=False # consider any dip that has at least one of the interesting movies

# ↗ Start here! ↖

<br/>
_Now go to `Cell`, and click `Run All`..._


In [3]:
# hit kimono-made Cinestar showtimes API
api_endpoint = 'http://www.kimonolabs.com/api/ondemand/9q0fwoh2?kimmodify=1&date=' + str(days_from_now)
results = json.load(urllib.urlopen(api_endpoint))

In [4]:
if not results['results']:
    display(Markdown('## _Error:_ Either there are no showings for your day, or you\'ve hit the API too many times...'+\
                     ' Try changing `days_from_now` and re-running?...'))
    print results

kino_program = double_dip.make_showtimes(results,
                                         ads_mins=ads_mins,
                                         trailers_mins=trailers_mins)

In [5]:
double_dips = \
double_dip.find_all_double_dips(kino_program,
                                max_waiting_mins=max_waiting_mins,
                                acceptable_overlap_mins=acceptable_overlap_mins,
                                movies_to_exclude=movies_to_exclude,
                                interesting_movies=interesting_movies,
                                all_interesting_movies_must_be_in_dip=all_interesting_movies_must_be_in_dip)

# Double Dips:

In [6]:
tmp = [display(Markdown('- ' + dip)) for dip in double_dips] 

- Hotel Transsilvanien 2 (OV) (13:30-15:20) -> Spectre (OV) (15:00-17:48) -> Irrational Man (OV) (17:30-19:25) -> Steve Jobs (OV) (19:30-21:53)

- Hotel Transsilvanien 2 (OV) (13:30-15:20) -> Spectre (OV) (15:00-17:48) -> Irrational Man (OV) (17:30-19:25) -> Macbeth (OV) (19:30-21:43)

- Irrational Man (OV) (17:30-19:25) -> Steve Jobs (OV) (19:30-21:53) -> Spectre (OV) (21:45-00:33)

- Irrational Man (OV) (17:30-19:25) -> Macbeth (OV) (19:30-21:43) -> Spectre (OV) (21:45-00:33)

- Irrational Man (OV) (20:20-22:15) -> Spectre (OV) (22:30-01:18)

...

- Spectre (OV) (20:00-22:48) -> Sicario (OV) (23:00-01:22)

- Spectre (OV) (20:45-23:33)

- Spectre (OV) (21:45-00:33)

- Spectre (OV) (22:30-01:18)

- Spectre (OV) (23:00-01:48)

In [7]:
kino_date = datetime.datetime.today() + datetime.timedelta(days=days_from_now)
display(Markdown('# kino program for ' + datetime.datetime.strftime(kino_date, '%A, %D')))
kino_program

# kino program for Sunday, 11/15/15

,movie_end_time,movie_start_time,start_time,title
0,1900-01-01 15:20:00,1900-01-01 13:50:00,1900-01-01 13:30:00,Hotel Transsilvanien 2 (OV)
1,1900-01-01 19:25:00,1900-01-01 17:50:00,1900-01-01 17:30:00,Irrational Man (OV)
2,1900-01-01 22:15:00,1900-01-01 20:40:00,1900-01-01 20:20:00,Irrational Man (OV)
3,1900-01-01 15:49:00,1900-01-01 13:50:00,1900-01-01 13:30:00,Crimson Peak (OV)
4,1900-01-01 18:53:00,1900-01-01 16:50:00,1900-01-01 16:30:00,Steve Jobs (OV)
5,1900-01-01 21:53:00,1900-01-01 19:50:00,1900-01-01 19:30:00,Steve Jobs (OV)
6,1900-01-02 01:08:00,1900-01-01 23:05:00,1900-01-01 22:45:00,Steve Jobs (OV)
7,1900-01-01 16:14:00,1900-01-01 13:50:00,1900-01-01 13:30:00,Der Marsianer - Rettet Mark Watney (OV)
8,1900-01-01 22:14:00,1900-01-01 19:50:00,1900-01-01 19:30:00,Der Marsianer - Rettet Mark Watney (OV)
9,1900-01-02 01:23:00,1900-01-01 23:20:00,1900-01-01 23:00:00,Black Mass (OV)
